In [11]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from hyperopt import hp
from ray.tune.search.bayesopt import BayesOptSearch
from ray.tune.schedulers import ASHAScheduler
from functools import partial
from ray.experimental.tqdm_ray import tqdm

import torch
from ray import train, tune
from ray.tune.search.optuna import OptunaSearch
from ray import train

from sklearn.model_selection import TimeSeriesSplit
import math
import matplotlib.pyplot as plt
import talib
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split

In [12]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("gpu")
else:
    device = torch.device('cpu')
print(torch.__version__)
print('CUDA available:', torch.cuda.is_available())
print('CUDA version:', torch.version.cuda)
print('cuDNN version:', torch.backends.cudnn.version())

gpu
2.1.2+cu121
CUDA available: True
CUDA version: 12.1
cuDNN version: 8902


In [13]:
def add_technical_indicators(spread, timeperiod=22):

    # MACD
    macd, macdsignal, macdhist = talib.MACD(spread, fastperiod=12, slowperiod=26, signalperiod=9)

    rsi = talib.RSI(spread, timeperiod=14)

    # CMO
    cmo = talib.CMO(spread, timeperiod=timeperiod)

    # MOM
    mom = talib.MOM(spread, timeperiod=timeperiod)

    # Bollinger Bands
    upperband, middleband, lowerband = talib.BBANDS(spread, timeperiod=timeperiod, nbdevup=2, nbdevdn=2, matype=0)

    # SMA
    sma = talib.SMA(spread, timeperiod=timeperiod)

    # Combine all indicators into a DataFrame
    indicators = pd.DataFrame({
        'MACD': macd,
        'MACD_signal': macdsignal,
        'MACD_hist': macdhist,
        'RSI': rsi,
        'CMO': cmo,
        'MOM': mom,
        'Upper_BB': upperband,
        'Middle_BB': middleband,
        'Lower_BB': lowerband,
        'SMA': sma
    })
    return indicators


In [14]:
# Load and preprocess data
data = pd.read_csv("Technology_Firm_Stock_Price.csv")
data = data.sort_values(by="Date")
data[760:]

,Date,ADSK,ORCL,PTC,CDNS,FICO,CRM,SNPS,INTU,TYL,...,NOW,MSFT,PANW,CDW,NTAP,STX,HPQ,WDC,AAPL,HPE
760,2023-01-09,192.470001,85.116173,125.959999,165.919998,593.869995,147.100006,325.950012,392.679321,318.989990,...,378.559998,225.123840,135.080002,181.744415,62.101120,53.098518,27.627516,37.590000,129.426544,16.445816
761,2023-01-10,193.429993,85.194977,127.400002,164.320007,585.859985,147.440002,327.630005,386.064148,313.070007,...,380.839996,226.838638,134.190002,184.135269,62.500202,54.173157,28.061609,38.169998,130.003326,16.542788
762,2023-01-11,199.929993,87.263283,127.209999,168.470001,598.969971,144.899994,334.350006,392.649536,318.089996,...,393.779999,233.697815,136.710007,188.284683,62.733810,54.892776,27.810801,38.419998,132.748016,16.397335
763,2023-01-12,199.960007,87.440559,131.089996,168.059998,605.580017,149.600006,333.100006,394.030212,316.559998,...,413.260010,236.413727,136.600006,191.060837,62.344467,55.430096,27.540699,37.959999,132.668442,15.825221
764,2023-01-13,200.589996,87.854233,129.690002,169.259995,617.919983,149.509995,332.970001,390.563690,316.760010,...,414.890015,237.127411,138.860001,192.503265,62.938221,54.940750,26.923326,37.220001,134.010941,15.951282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,2023-12-22,242.759995,105.790077,173.970001,275.820007,1168.880005,266.339996,524.460022,623.131348,415.000000,...,697.549988,374.579987,298.209991,226.630005,88.882095,85.160004,30.180000,52.660000,193.600006,17.090000
1002,2023-12-26,245.070007,105.780121,175.309998,274.959991,1164.619995,266.220001,520.250000,623.910156,416.410004,...,701.229980,374.660004,300.820007,228.729996,88.862213,85.870003,30.410000,52.759998,193.050003,17.230000
1003,2023-12-27,245.110001,105.531090,175.720001,274.640015,1170.609985,266.720001,518.099976,628.173767,415.600006,...,703.760010,374.070007,297.500000,228.550003,88.086731,85.680000,30.240000,52.419998,193.149994,17.090000
1004,2023-12-28,244.910004,105.839890,175.919998,273.239990,1169.339966,265.579987,517.409973,627.075439,418.290009,...,702.460022,375.279999,295.579987,228.699997,87.698990,86.790001,30.180000,52.930000,193.580002,17.170000


In [15]:
selected_pairs = [('IBM', 'AAPL'), ('KEYS', 'MSI'), ('TDY', 'ROP')]

spreads_with_indicators = {}

for stock1, stock2 in selected_pairs:
    spread = data[stock1] - data[stock2]
    indicators = add_technical_indicators(spread)
    indicators['Spread'] = spread
    spreads_with_indicators[(stock1, stock2)] = indicators.dropna()  

spreads_with_indicators

{('IBM',
  'AAPL'):           MACD  MACD_signal  MACD_hist        RSI        CMO        MOM  \
 33    2.718908     2.943616  -0.224708  58.515017  16.396557  10.806602   
 34    2.666835     2.888260  -0.221425  60.005093  18.344872  10.280739   
 35    2.669472     2.844502  -0.175030  62.018439  20.988334   7.776169   
 36    2.539152     2.783432  -0.244280  57.698306  15.574670   7.671303   
 37    2.195091     2.665764  -0.470673  49.880805   5.265781   6.590240   
 ...        ...          ...        ...        ...        ...        ...   
 1001  0.121955     0.203806  -0.081851  55.195276   7.504150   5.269989   
 1002  0.399330     0.242911   0.156419  58.508264  11.819718   6.339996   
 1003  0.624062     0.319141   0.304921  58.811969  12.217028   5.100021   
 1004  0.781853     0.411683   0.370170  58.382420  11.724800   4.389984   
 1005  0.964375     0.522222   0.442154  60.279122  14.120591   5.770004   
 
        Upper_BB  Middle_BB   Lower_BB        SMA     Spread  
 33 

In [16]:
def rolling_window(data):
    time_step = 22

    X, y = [], []

    for i in range(len(data) - time_step):
        X.append(data.iloc[i : (i+time_step)])
        y.append([data.iloc[i+time_step, -1]])

    X = np.array(X)
    y = np.array(y)

    print(X.shape)
    print(y.shape)

    return X, y

In [17]:
spreads_with_indicators[('IBM','AAPL')].isna().sum()

MACD           0
MACD_signal    0
MACD_hist      0
RSI            0
CMO            0
MOM            0
Upper_BB       0
Middle_BB      0
Lower_BB       0
SMA            0
Spread         0
dtype: int64

In [18]:
spreads_with_indicators[('IBM','AAPL')]

,MACD,MACD_signal,MACD_hist,RSI,CMO,MOM,Upper_BB,Middle_BB,Lower_BB,SMA,Spread
33,2.718908,2.943616,-0.224708,58.515017,16.396557,10.806602,48.650186,38.422339,28.194491,38.422339,41.752815
34,2.666835,2.888260,-0.221425,60.005093,18.344872,10.280739,48.864307,38.889645,28.914983,38.889645,42.428154
35,2.669472,2.844502,-0.175030,62.018439,20.988334,7.776169,49.274190,39.243107,29.212025,39.243107,43.353630
36,2.539152,2.783432,-0.244280,57.698306,15.574670,7.671303,49.457774,39.591803,29.725831,39.591803,42.075424
37,2.195091,2.665764,-0.470673,49.880805,5.265781,6.590240,49.305938,39.891359,30.476781,39.891359,39.405037
...,...,...,...,...,...,...,...,...,...,...,...
1001,0.121955,0.203806,-0.081851,55.195276,7.504150,5.269989,-28.950662,-33.109091,-37.267519,-33.109091,-31.460007
1002,0.399330,0.242911,0.156419,58.508264,11.819718,6.339996,-28.674972,-32.820909,-36.966846,-32.820909,-29.839996
1003,0.624062,0.319141,0.304921,58.811969,12.217028,5.100021,-28.340423,-32.589090,-36.837756,-32.589090,-29.689987
1004,0.781853,0.411683,0.370170,58.382420,11.724800,4.389984,-28.054526,-32.389545,-36.724564,-32.389545,-29.830002


In [19]:
scaled_spreads_with_indicators = {}  

for key, dataframe in spreads_with_indicators.items():

    scaled_df = pd.DataFrame()
    scaler_dict = {}

    for column in dataframe.columns:
        scaler = MinMaxScaler(feature_range=(0, 1)) 
        scaled_values = scaler.fit_transform(dataframe[column].values.reshape(-1, 1))

        scaler_dict[column] = scaler
        scaled_df[column] = scaled_values.flatten()

    X, y = rolling_window(scaled_df)
    scaled_spreads_with_indicators[key] = {"data": scaled_df, "scaler_obj": scaler_dict["Spread"], "X": X, "y": y}

scaled_spreads_with_indicators[('IBM', 'AAPL')]["data"].isna().sum()


(951, 22, 11)
(951, 1)
(951, 22, 11)
(951, 1)
(951, 22, 11)
(951, 1)


MACD           0
MACD_signal    0
MACD_hist      0
RSI            0
CMO            0
MOM            0
Upper_BB       0
Middle_BB      0
Lower_BB       0
SMA            0
Spread         0
dtype: int64

In [20]:
scaled_spreads_with_indicators[('IBM', 'AAPL')]["y"]

array([[0.78915659],
       [0.81670677],
       [0.82245154],
       [0.84476828],
       [0.83335498],
       [0.8531988 ],
       [0.83979511],
       [0.82761547],
       [0.85400659],
       [0.83523632],
       [0.84979239],
       [0.85748264],
       [0.87428388],
       [0.88607884],
       [0.87173508],
       [0.86089984],
       [0.82874729],
       [0.80211689],
       [0.84281954],
       [0.85809332],
       [0.85069642],
       [0.85194083],
       [0.86920119],
       [0.87594857],
       [0.88424179],
       [0.89708301],
       [0.89419683],
       [0.85778537],
       [0.84150968],
       [0.83095507],
       [0.82764616],
       [0.82506085],
       [0.81579893],
       [0.81258272],
       [0.79865839],
       [0.78955062],
       [0.7645608 ],
       [0.76931852],
       [0.77365428],
       [0.79116675],
       [0.78588254],
       [0.78023811],
       [0.76890406],
       [0.75892857],
       [0.78866383],
       [0.81346755],
       [0.80569626],
       [0.809

In [21]:
class RollingWindowDataset(Dataset):
    def __init__(self, X, y, device="gpu"):
        self.X = torch.tensor(X, dtype=torch.float, device=device)
        self.y = torch.tensor(y, dtype=torch.float, device=device)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx].clone().detach().to(torch.float), self.y[idx].clone().detach().to(torch.float)


In [22]:
class CNNLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, conv_channels, kernel_size, pool_size):
        super(CNNLSTMModel, self).__init__()

        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        # Convolutional Layer
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=conv_channels, kernel_size=(kernel_size,))
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool1d(kernel_size=pool_size)

        conv_output_length = (input_dim - kernel_size + 1)
        pool_output_length = (conv_output_length - 2) + 1

        # The input_size for LSTM should be the number of features for each time step after pooling
        new_input_dim = conv_channels  

        # LSTM Layer
        self.lstm = nn.LSTM(input_size=new_input_dim, hidden_size=hidden_dim, num_layers=layer_dim, batch_first=True)

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Convolutional layer forward
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = x.transpose(1, 2)  # Adjust dimensions if needed

        assert x.size(-1) == self.lstm.input_size, f"Mismatch in LSTM input size. Expected: {self.lstm.input_size}, Got: {x.size(-1)}"

        # Initializing hidden state for first input
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(device)

        # Initializing cell state for first input
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(device)

        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        out = self.fc(out[:, -1, :])

        return out

In [23]:
X_train, X_test, y_train, y_test = train_test_split(scaled_spreads_with_indicators[('IBM', 'AAPL')]["X"], scaled_spreads_with_indicators[('IBM', 'AAPL')]["y"], test_size=0.2, shuffle=False)
X_train.shape
y_train.shape
print(f"x_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")

print(f"x_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")

x_train: (760, 22, 11)
y_train: (760, 1)
x_test: (191, 22, 11)
y_test: (191, 1)


In [24]:
train_data = RollingWindowDataset(X_train, y_train, device)
test_data = RollingWindowDataset(X_test, y_test, device)

In [25]:
model = CNNLSTMModel(input_dim=X_train.shape[2], hidden_dim=22, layer_dim=1, output_dim=y_train.shape[1], conv_channels=1, kernel_size=3, pool_size=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.MSELoss()

In [26]:
class CNNLSTM_Trainer:
    def __init__(self, train_data, test_data, device, optimizer, criterion):
        self.train_data = train_data
        self.test_data = test_data
        self.device = device
        self.optimizer = optimizer
        self.criterion =  criterion
        self.model = CNNLSTMModel(input_dim=11, hidden_dim=22, layer_dim=1, output_dim=1, conv_channels=16, kernel_size=3, pool_size=2).to(device)

    def train_lstm(self, config, checkpoint_dir=None):

        batch_size = int(round(config["batch_size"]))
        epochs = int(round(config["epochs"]))
        hidden_dim = int(round(config["hidden_dim"]))
        layer_dim = int(round(config["layer_dim"]))
        learning_rate = config["learning_rate"]
        kernel_size = config["kernel_size"]
        conv_channels = config["conv_channels"]
        pool_size = config["pool_size"]

        # Create the model
        self.model = CNNLSTMModel(input_dim=11, hidden_dim=hidden_dim, layer_dim=layer_dim, output_dim=1, conv_channels=conv_channels, kernel_size=kernel_size, pool_size=pool_size).to(device)

        # Define loss and optimizer
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

        # Define data loader (you may need to adapt this based on your data format)
        train_loader = DataLoader(self.train_data, batch_size=batch_size, shuffle=False)

        # Training loop
        for epoch in range(epochs):
            print('epochs {}/{}'.format(epoch+1,epochs))
            running_loss = .0
            self.model.train()


            for batch_idx, (data, target) in enumerate(train_loader):
                if isinstance(data, np.ndarray):
                    data = torch.from_numpy(data).float()
                if isinstance(target, np.ndarray):
                    target = torch.from_numpy(target).float()

                data, target = data.to(self.device), target.to(self.device)

                self.optimizer.zero_grad()
                predictions = self.model(data)
                loss = self.criterion(predictions, target)
                loss.backward()
                self.optimizer.step()

                running_loss += loss.item()


            train_loss = running_loss/len(train_loader)
            print(train_loss)
            train.report({'mse': train_loss})


    def Test(self, config, test_data):
        batch_size = int(round(config["batch_size"]))
        self.test_data = test_data

        all_predictions = []
        test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

        running_loss = .0

        self.model.eval()

        with torch.no_grad():

            for batch_idx, (data, target) in enumerate(test_loader):
                if isinstance(data, np.ndarray):
                    data = torch.from_numpy(data).float()
                if isinstance(target, np.ndarray):
                    target = torch.from_numpy(target).float()

                data, target = data.to(self.device), target.to(self.device)

                self.optimizer.zero_grad()
                preds = self.model(data)
                loss = self.criterion(preds,target)
                running_loss += loss.item()

                all_predictions.extend(preds.cpu().numpy())


            test_loss = running_loss/len(test_loader)

            print(f'test_loss {test_loss}')

        all_predictions = np.array(all_predictions)

        return all_predictions


    def get_current_model(self):
        if self.model is not None:
            return self.model
        else:
            raise ValueError("Model has not been trained yet.")


In [27]:
def train_model_with_config(config, train_data, test_data, device, optimizer, criterion):
    trainer = CNNLSTM_Trainer(train_data, test_data, device, optimizer, criterion)
    return trainer.train_lstm(config)

In [28]:
def tuning(train_data, test_data, device, optimizer, criterion):
    ray.shutdown()
    ray.init(num_cpus=8, num_gpus=1)

    # Define hyperparameter configuration
    config = {
        "learning_rate": tune.choice([1e-4, 1e-3, 1e-2]),
        "batch_size": tune.choice([32, 64, 128]),
        "epochs": tune.choice([50, 100, 150]),
        "hidden_dim": tune.choice([25, 50, 75, 100]),
        "layer_dim": tune.choice([1, 5, 10]),
        "kernel_size": tune.choice([3, 5, 7]),
        "pool_size": tune.choice([2, 3, 4]),
        "conv_channels": tune.choice([32, 64, 128])
    }

    # Define scheduler and reporter
    scheduler = ASHAScheduler(
        metric="mse",
        mode="min",
        grace_period=10,
        reduction_factor=3
    )
    reporter = CLIReporter(metric_columns=["mse", "training_iteration"])

    # Optuna search algorithm
    optuna_search = OptunaSearch(metric="mse", mode="min")

    # Partial function for training with LSTM and given data
    #train_lstm_with_data = partial(trainer.train_lstm, device=trainer.device)

    trainable_with_cpu_gpu = tune.with_resources(lambda config: train_model_with_config(config, train_data, test_data, device, optimizer, criterion), {"cpu": 8, "gpu": 1})


    # Start the tuning process
    analysis = tune.run(
        trainable_with_cpu_gpu,
        # resources_per_trial={"cpu": 8, "gpu": 1},
        config=config,
        num_samples=10,
        scheduler=scheduler,
        search_alg=optuna_search,
        progress_reporter=None,
        verbose=0
    )

    # Get the best hyperparameters
    best_config = analysis.get_best_config(metric="mse", mode="min")
    best_mse = analysis.get_best_trial(metric="mse", mode="min").last_result["mse"]

    print("Best config: ", best_config)
    print(f"Best MSE: {best_mse}")

    return best_config

In [29]:
signal_dict = {}
empty_df = pd.DataFrame(columns=["signal", "pred", "prev_spread", "Next_spread"])



for key, dataframe in scaled_spreads_with_indicators.items():
    X_train, X_test, y_train, y_test = train_test_split(scaled_spreads_with_indicators[key]["X"], scaled_spreads_with_indicators[key]["y"], test_size=0.2, shuffle=False)
    records = []

    X_train = X_train.transpose(0, 2, 1)
    X_test = X_test.transpose(0, 2, 1)
    
    train_data = RollingWindowDataset(X_train, y_train, device)
    test_data = RollingWindowDataset(X_test, y_test, device)

    best_config = tuning(train_data, test_data, device, optimizer, criterion)
    trainer = CNNLSTM_Trainer(train_data, test_data, device, optimizer, criterion)
    trainer.train_lstm(best_config)
    preds = trainer.Test(best_config, test_data)
    mape = mean_absolute_percentage_error(y_test, preds)*100

    for i in range(len(X_test)):
        df = scaled_spreads_with_indicators[key]["data"]
        prev_spread = df.iloc[781+i]["Spread"]

        #assert df.iloc[781+i] == X_test[i][10][21], f"Mismatch in LSTM input size. Expected: {df.iloc[781+i]}, Got: {X_test[i][10][21]}"
        signal = 1 if preds[i][0] > prev_spread else 0

        records.append({
        "signal": signal, 
        "pred": preds[i][0], 
        "prev_spread": prev_spread, 
        "next_spread": y_test[i]
        })
    
    empty_df = pd.DataFrame(records)

    signal_dict[key] = empty_df


    

(<lambda> pid=467177) epochs 1/50
(<lambda> pid=467177) 0.30026182741858065
(<lambda> pid=467177) epochs 2/50
(<lambda> pid=467177) 0.21812662350324294
(<lambda> pid=467177) epochs 3/50
(<lambda> pid=467177) 0.10578390851151198
(<lambda> pid=467177) epochs 4/50
(<lambda> pid=467177) 0.025856534610890474
(<lambda> pid=467177) epochs 5/50
(<lambda> pid=467177) 0.03659687624895014
(<lambda> pid=467177) epochs 6/50
(<lambda> pid=467177) 0.029697410568284493
(<lambda> pid=467177) epochs 7/50
(<lambda> pid=467177) 0.031099574131076224
(<lambda> pid=467177) epochs 8/50
(<lambda> pid=467177) 0.030200563264467444
(<lambda> pid=467177) epochs 9/50
(<lambda> pid=467177) 0.029943531631336857
(<lambda> pid=467177) epochs 10/50
(<lambda> pid=467177) 0.029562398039464217
(<lambda> pid=467177) epochs 11/50
(<lambda> pid=467177) 0.02917541873466689
(<lambda> pid=467177) epochs 12/50
(<lambda> pid=467177) 0.028769060746223356
(<lambda> pid=467177) epochs 13/50
(<lambda> pid=467177) 0.02833095228803965
(

In [34]:
import os

# Specify the directory where you want to save the CSV files
output_dir = '/home/arda/Cs_440/Statistical_Arbitrage/Statistical-Arbitrage/'
os.makedirs(output_dir, exist_ok=True)

for key, value in signal_dict.items():
    # Define the file path using the key
    file_path = os.path.join(output_dir, f"{key}.csv")
    
    # Save the DataFrame to a CSV file
    value.to_csv(file_path, index=False)
    
    # Print the saved file path
    print(f"Saved {key}.csv to {output_dir}")

Saved ('IBM', 'AAPL').csv to /home/arda/Cs_440/Statistical_Arbitrage/Statistical-Arbitrage/
Saved ('KEYS', 'MSI').csv to /home/arda/Cs_440/Statistical_Arbitrage/Statistical-Arbitrage/
Saved ('TDY', 'ROP').csv to /home/arda/Cs_440/Statistical_Arbitrage/Statistical-Arbitrage/


In [31]:
trainer = CNNLSTM_Trainer(train_data, test_data, device, optimizer, criterion)
trainer.train_lstm(best_config)

epochs 1/100
0.21924530990266552
epochs 2/100
0.04341929010115564
epochs 3/100
0.017694259722096223
epochs 4/100
0.013239183792999635
epochs 5/100
0.012075027043465525
epochs 6/100
0.010562695698657384
epochs 7/100
0.010396058341333022
epochs 8/100
0.009780226061896732
epochs 9/100
0.009215090365614742
epochs 10/100
0.008892941094624499
epochs 11/100
0.008407402395581206
epochs 12/100
0.008086109010037035
epochs 13/100
0.007789498641310881
epochs 14/100
0.007507189273989449
epochs 15/100
0.007260664288575451
epochs 16/100
0.00705936091253534
epochs 17/100
0.006897993075350921
epochs 18/100
0.00676585896871984
epochs 19/100
0.006646657110347102
epochs 20/100
0.0065495889478673535
epochs 21/100
0.006466498462638508
epochs 22/100
0.006392768836424996
epochs 23/100
0.006319664030646284
epochs 24/100
0.0062484701241677
epochs 25/100
0.006183881642452131
epochs 26/100
0.006117753257664542
epochs 27/100
0.006055981755101432
epochs 28/100
0.005990000441670418
epochs 29/100
0.005934310902375728

In [32]:
preds = trainer.Test(best_config, test_data)
mape = mean_absolute_percentage_error(y_test, preds)*100
print(mape)

test_loss 0.007333235892777641
180547913361385.0
